<a href="https://colab.research.google.com/github/elixir-biohackathon-project-12/network-analysis/blob/main/get_collector_gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check for gender

## Init and load data

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('id_list').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame.from_records(rows)

# SPARQLWrapper
!pip install -q SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON

[['http://www.wikidata.org/entity/Q100156193'], ['http://www.wikidata.org/entity/Q100156193'], ['http://www.wikidata.org/entity/Q100157099'], ['http://www.wikidata.org/entity/Q100587885'], ['http://www.wikidata.org/entity/Q100600540'], ['http://www.wikidata.org/entity/Q100694962'], ['http://www.wikidata.org/entity/Q100699957'], ['http://www.wikidata.org/entity/Q100771006'], ['http://www.wikidata.org/entity/Q100783450'], ['http://www.wikidata.org/entity/Q100876'], ['http://www.wikidata.org/entity/Q100919649'], ['http://www.wikidata.org/entity/Q100920290'], ['http://www.wikidata.org/entity/Q100935946'], ['http://www.wikidata.org/entity/Q100939400'], ['http://www.wikidata.org/entity/Q101069260'], ['http://www.wikidata.org/entity/Q101069270'], ['http://www.wikidata.org/entity/Q101069287'], ['http://www.wikidata.org/entity/Q101069300'], ['http://www.wikidata.org/entity/Q101069359'], ['http://www.wikidata.org/entity/Q101069427'], ['http://www.wikidata.org/entity/Q101069525'], ['http://www.wi

## Loop over de wd IDs

In [ ]:
for row in rows:
  url = str(row[0])
  if 'http://www.wikidata.org/entity/' in url:
    item = url.replace('http://www.wikidata.org/entity/', 'wd:')
    s_query = """
    SELECT ?label ?gender ?genderLabel ?birthdate ?deaddate ?country ?countryLabel WHERE {{
      {0} rdfs:label ?label .
      OPTIONAL{{ {0} wdt:P21 ?gender .}}.
      OPTIONAL{{ {0} wdt:P569 ?birthdate .}}.
      OPTIONAL{{ {0} wdt:P570 ?deaddate .}}.
      OPTIONAL{{ {0} wdt:P27 ?country .}}.
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}

    """.format(item)
    sparql = SPARQLWrapper('https://query.wikidata.org/sparql')
    sparql.setQuery(s_query)
    sparql.setReturnFormat(JSON)

    results = sparql.query().convert()
    try:
      result = results["results"]["bindings"][0]
      name = result['label']['value']
      try:
        gender = result['genderLabel']['value']
      except KeyError:
        gender = 'unknown'
      try:
        country = result['countryLabel']['value']
      except KeyError:
        country = 'unknown'
      try:
        birth = result['birthdate']['value']
      except KeyError:
        birth = 'S.D.'
      try:
        dead = result['deaddate']['value']
      except KeyError:
        dead = 'S.D.'
    except IndexError:
      pass
  
  row.append(name)
  row.append(gender)
  row.append(country)
  row.append(birth)
  row.append(dead)


print(rows)

[['http://www.wikidata.org/entity/Q100156193', 'Laurence Henry Millener', 'male', 'New Zealand', '1914-04-01T00:00:00Z', '2000-01-01T00:00:00Z'], ['http://www.wikidata.org/entity/Q100156193', 'Laurence Henry Millener', 'male', 'New Zealand', '1914-04-01T00:00:00Z', '2000-01-01T00:00:00Z'], ['http://www.wikidata.org/entity/Q100157099', 'Ross Henry Michie', 'male', 'New Zealand', '1894-01-01T00:00:00Z', '1987-01-01T00:00:00Z'], ['http://www.wikidata.org/entity/Q100587885', 'Myrtle L. T. Adams', 'female', 'United States of America', '1903-12-13T00:00:00Z', '1989-02-16T00:00:00Z'], ['http://www.wikidata.org/entity/Q100600540', 'Hubert Walter Simmonds', 'unknown', 'unknown', '1877-01-01T00:00:00Z', '1966-03-16T00:00:00Z'], ['http://www.wikidata.org/entity/Q100694962', 'Leonora Gloyd', 'female', 'United States of America', '1902-08-29T00:00:00Z', '1993-06-02T00:00:00Z'], ['http://www.wikidata.org/entity/Q100699957', 'Lewis J. Stannard', 'male', 'United States of America', '1918-04-15T00:00:0

In [ ]:
df_2 = pd.DataFrame.from_records(rows)

print(df_2)

                                              0  ...                     5
0     http://www.wikidata.org/entity/Q100156193  ...  2000-01-01T00:00:00Z
1     http://www.wikidata.org/entity/Q100156193  ...  2000-01-01T00:00:00Z
2     http://www.wikidata.org/entity/Q100157099  ...  1987-01-01T00:00:00Z
3     http://www.wikidata.org/entity/Q100587885  ...  1989-02-16T00:00:00Z
4     http://www.wikidata.org/entity/Q100600540  ...  1966-03-16T00:00:00Z
...                                         ...  ...                   ...
3005      https://orcid.org/0000-0003-4859-5149  ...  2002-01-01T00:00:00Z
3006      https://orcid.org/0000-0003-4881-3636  ...  2002-01-01T00:00:00Z
3007      https://orcid.org/0000-0003-4888-2578  ...  2002-01-01T00:00:00Z
3008      https://orcid.org/0000-0003-4989-2014  ...  2002-01-01T00:00:00Z
3009      https://orcid.org/0000-0003-4994-3897  ...  2002-01-01T00:00:00Z

[3010 rows x 6 columns]


In [ ]:
from gspread_dataframe import set_with_dataframe
title = 'collectors_info_v2'
gc.create(title)  # if not exist
sheet = gc.open(title).sheet1
set_with_dataframe(sheet, df_2) 